başlangıçta client ile japonyadaki bir depremin verisini kullandım inventory okumayı fıltrelemeyı ve matplotlib kullanmayı bu data üzerinde denedim. 
İkinci kısımda 6.8 gaziantep depremini kullanarak response removal yapmayı denedim. 
anlamadığım kısımları her bölümün başında not olarak yazdım.     

In [ ]:
import obspy
from obspy import read_inventory,UTCDateTime
from obspy.clients.fdsn import Client
from obspy.io.xseed import Parser
from obspy.signal import PPSD
import matplotlib.pyplot as plt

In [ ]:
client= Client("IRIS")
t1 = UTCDateTime("2020-01-02T18:23:53.011303Z")
t2=t1+300 #5 dakika
st= client.get_waveforms("G", "INU", "00", "BHZ", t1, t2)


In [ ]:
print(st[0].stats)
print(st.plot())

japonya depreminin inventory si

In [ ]:
inventory= read_inventory(r"/home/master/Desktop/Workspace/EarthML/obspy/data/inv_japan.xml")
print(inventory[0][0][0].response)
inventory.plot_response(min_freq=0.001)#hangi frekanstan itibaren plotlanacağını belirliyor
plt.show()

In [ ]:
print("Displacement")
inventory.plot_response(min_freq=0.001,channel="BHZ", output="DISP")
print("Velocity")
inventory.plot_response(0.001,channel="BHZ", output="VEL")
print("Acceleration")
inventory.plot_response(0.001,channel="BHZ", output="ACC")
plt.show()

one tick per line neyi değiştiriyor anlayamadım 

In [ ]:
print(st[0].plot(type="dayplot", interval=3, right_vertical_labels=True,
vertical_scaling_range=6e5, one_tick_per_line=True,
show_y_UTC_label=False,
events={'min_magnitude': 4}))

matplotlib

In [ ]:

fig=plt.figure(figsize=(6,4))
ax=fig.add_subplot(1,1,1)
ax.plot(st[0].times('matplotlib'),st[0].data)
ax.xaxis_date()
fig.autofmt_xdate() #times('matplotlib) tarihlerini düzeltmek için gerekli
plt.show()

times('matplotlib') kullandıktan sonra tarihler halen doğru formatta olmuyor  

In [ ]:
print(st[0].times(),'\n')
print(st[0].times('matplotlib'))

filtering

In [ ]:
st_sliced=st.slice(starttime=st[0].stats.starttime+60,endtime=st[0].stats.endtime-30)
st_sliced_copy=st_sliced.copy()
print(st_sliced_copy[0].data[2000:2050]) #datanın bir kısmı
print(st_sliced_copy[0].plot())
st_sliced_copy[0].spectrogram(log=True,cmap='inferno')
st_sliced.normalize()
st_sliced.detrend(type='constant') #polynomial linear ve simple da var
st_sliced.taper(0.05,type="hann") #başlangıç ve bitişleri aynı faza getirmek için?
print(st_sliced[0].plot())
st_sliced[0].spectrogram(log=True, cmap='inferno', dbscale=False)
print(st_sliced[0].data[2000:2050])

low high ve band pass (corners ne değiştiriyor anlamadım)

In [ ]:
tr_filt=st[0].copy().detrend('constant')
tr_filt.filter('lowpass',freq=0.5,corners=5,zerophase=True)
st[0].plot()
print(tr_filt.plot())

In [ ]:
tr_filt=st[0].copy().detrend('constant').slice(starttime=st[0].stats.starttime+200,endtime=st[0].stats.starttime+230)
for i in range (1):
    tr_filt.filter('lowpass',freq=0.5,corners=1,zerophase=True)
#st[0].plot()
print(tr_filt.plot())

In [ ]:
tr_filt=st[0].copy().detrend('constant')
tr_filt.filter('highpass',freq=2.0,corners=5,zerophase=True)
st[0].plot()
print(tr_filt.plot())

In [ ]:
tr_filt=st[0].copy().detrend('constant')
tr_filt.filter('bandpass',freqmin=0.5,freqmax=2.0,corners=5,zerophase=True)
st[0].plot()
print(tr_filt.plot())

downsampling (decimate)

In [ ]:
import numpy as np
from obspy import Trace, Stream
tr = Trace(data=np.arange(10))
st2 = Stream(traces=[tr])
tr.stats.sampling_rate
print(tr.data)
tr.plot()
st2.decimate(4, strict_length=False, no_filter=True)
tr.stats.sampling_rate
print(tr.data)
print(tr.plot())

In [ ]:
st_=st.copy()
ti=st_[0].stats.starttime+80
tf=st_[0].stats.starttime+120
st_.plot(starttime=ti,endtime=tf)
st_.decimate(factor=4,strict_length=False,no_filter=True)#burada no filter neyi değiştiriyor?
print(st_.plot(starttime=ti,endtime=tf))

In [ ]:
#orijinal ve downsapled datanın filtrelenmesinin karşılaştırılması
def filt(st):
    tr_filt=st[0].copy()
    tr_filt.detrend('constant')
    tr_filt.filter('lowpass',freq=1.0,corners=5,zerophase=True)
    st[0].plot(starttime=ti,endtime=tf)
    print(tr_filt.plot(starttime=ti,endtime=tf))
filt(st)    
filt(st_)

ppsd (ne olduğunu anlamadım) 

In [ ]:
from obspy.signal import PPSD
tr_p=st.select(id="G.INU.00.BHZ")[0]
print(tr_p)
print(inventory)
ppsd = PPSD(tr_p.stats, metadata=inventory,ppsd_length=300.0)
ppsd.add(st)
ppsd.plot()

buradan sonra response removalla ilgili başka bir data üzerinden birşeyler denedim

In [ ]:
import obspy
from obspy import read,read_inventory,UTCDateTime
from obspy.clients.fdsn import Client
from obspy.io.xseed import Parser
from obspy.signal import PPSD
import matplotlib.pyplot as plt

In [ ]:
st=read(r'/home/master/Desktop/Workspace/EarthML/obspy/data/erz.mseed')
tr=st.select(component="Z")[0]
inv= read_inventory(r"/home/master/Desktop/Workspace/EarthML/obspy/data/erz_inv.xml")
print(tr)
print(inv)
print(inv[0][0][0].response)

burada plot_response dediğimizde velocity grafiğini çiziyor output="vel" belirtilmemesine rağmen. Bunun nedeni ne? 

In [ ]:
inv.plot_response(min_freq=0.001,channel="BHZ")
plt.show()

In [ ]:
print(tr.stats)
tr.plot(color='red',bgcolor='black')
plt.show()

In [ ]:
st_new=st.copy().taper(0.10,type='hann').filter("bandpass", freqmin=0.005,freqmax=1.0)
st_new.plot(color='red',bgcolor='black')
plt.show()

bu kisimda sonda oluşan grafiğin düzgün olmasının sebebi overamplification mı oluyor?

In [ ]:
st_response_removed=st.copy().taper(0.10,type='hann').remove_response(inventory=inv,output="VEL")
st_response_removed.filter("bandpass", freqmin=0.005,freqmax=1,zerophase=True).plot(
    color='red',bgcolor='black')
plt.show()

In [ ]:
# Define frequency domains of datas 

freqs = np.fft.rfftfreq(st[0].stats.npts, d=st[0].stats.delta)
print(freqs)
print("Frequency domain for st is between",min(freqs),"and",max(freqs),"\n")

In [ ]:
pre_filt = (0.001, 0.005, 10.0, 20.0) #st nin frekans aralığından seçildi

In [ ]:
st_response_removed=st.copy().remove_response(
    inventory=inv,output="DISP",pre_filt=pre_filt,taper=True,
    taper_fraction=0.05,water_level=60, plot=True)[0]

In [ ]:
tr_p=st.select(id="KO.ILIC..BHZ")[0]
print(tr_p)
inventory=obspy.read_inventory(r'/home/master/Desktop/Workspace/EarthML/obspy/data/erz_ınv.xml')
print(inventory)
metadata=inventory.get_response(tr_p.id,tr_p.stats.starttime)
print(metadata)
ppsd = PPSD(tr_p.stats, metadata=inventory,ppsd_length=247.1)
ppsd.add(st)
ppsd.plot()

In [ ]:
print(st.select(id='KO.ILIC..BHZ'))
print(st)

trigger


In [ ]:
st.plot()
plt.show()

In [ ]:
i=1
from obspy.signal.trigger import plot_trigger, classic_sta_lta ,trigger_onset
df=st[i].stats.sampling_rate
cft=classic_sta_lta(st[i].data, int(5 * df), int(10 * df))
plot_trigger(st[i],cft,1.7,0.2)

In [ ]:
onof=trigger_onset(cft, 1.7,0.2)
print(onof.shape)

determining absolute time of triggers


In [ ]:
print(onof)

In [ ]:
st[0].stats

In [ ]:
sampling_rt=st[0].stats.sampling_rate
time=st[0].stats.starttime+(onof[0,0]/sampling_rt)
print(time)

In [ ]:
detection_times=[]
for i in np.arange(0, onof.shape[0]):
    time=st[0].stats.starttime+(onof[i,0]/sampling_rt)
    detection_times.append(time)

In [ ]:
detection_times

tamplate matching(sonra bakacağım)


In [ ]:
fs=1/1000
y = np.arange(0, 1+fs, fs)
s=np.concatenate([np.zeros(400),np.ones(100),np.zeros(501)])
plt.plot(y,s)
plt.ylim(0,2)
plt.show()

In [ ]:
import numpy as np
nl= 0.4
n= np.sqrt(nl)*(np.random.rand(1001))
r= s+n
plt.plot(y,r)
plt.ylim(0,2)

In [ ]:
import logging